In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import httplib2
import os
from os.path import exists
import csv
import copy
import import_ipynb
import nltk
from functions import *
import final_res as FS


importing Jupyter notebook from functions.ipynb
importing Jupyter notebook from final_res.ipynb


C:\Users\kasho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\kasho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.1 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
<string>:17: FutureWarning: The default v

In [ ]:

nltk.download('punkt')

In [6]:

links = open("../websites.txt" , "r")
result_path = "../result"

categories = {}
categories.update(dict.fromkeys(["cricket", "football", "tennis"] , "Sports"))
categories.update(dict.fromkeys([ "gulf", "americas", "mena", "asia"] , "World"))

categories_occurences = {
    "sports" : 0,
    "world"  : 0,
    "others" : 0
}

words = [" and ", " or ", " nor ", " if ", " because ", " unless "]
website_list = links.read()
website_list = website_list.split("\n")

### Links extractor

In [8]:
for link in website_list:
    url_extract = requests.get(link).text
    s = ""
    if link == "https://nypost.com/":
        link = link + "20"
    soup = BeautifulSoup(url_extract, )
    texts = soup.find_all('a')
    for L in texts:
        s+= str(L)
    link_splitted = link.split('/')
    website_name = link_splitted[2]
    website_name = website_name.replace("www.","")
    website_name = website_name.replace(".com","")
    s = set(s.split('"'))
    path = result_path + '/'+website_name
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(path + '/texts'):
        os.makedirs(path+ '/texts')
    outFile = open(path+"/"+website_name+"_Links.txt", "w")
    for x in s:
        if x.startswith(link) and '-' in x and len(x) >60 and not (".svg" in x or ".png" in x):
            outFile.write(x + "\n")

### Artical text extractor

In [14]:
for link in website_list:
    website_name = link.split('/')[2]
    website_name = website_name.replace("www.","")
    website_name = website_name.replace(".com","")
    path = result_path + '/'+website_name
    inFile = open(result_path + '/'+website_name+'/'+website_name+"_Links.txt", "r")
    links = inFile.readlines()
    for n in links:
        n = n.strip()
        url_extract = requests.get(n).text
        bool = 'true'
        soup = BeautifulSoup(url_extract, )
        texts = soup.find_all('p')
        s =  ''

        for L in texts:
            s+=  L.text + " "
        
        genre , chance = FS.predict_from_text(s)
        
        checking = open('checking.txt' , encoding='utf-8')
        checking.write("%a genre with chance %a", genre, chance)

        outFile = open(result_path + '/'+website_name+'/texts/'+ genre +".txt", "a+" , encoding='utf-8' )

        outFile.write('\n')
        for x in s:
            outFile.write(x)
            outFile.write(".\n")
    outFile.close()

<string>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
<string>:31: FutureWarning: The default value of regex will change from True to False in a future version.


In [23]:
import import_ipynb
import functions 
import imp
imp.reload(functions)
linkToText("https://www.khaleejtimes.com/uae/dust-storm-in-uae-visibility-down-to-100-metres-in-some-areas")


importing Jupyter notebook from functions.ipynb


'A dust storm has hit parts of Abu Dhabi, with visibility reduced to less than 100 metres in some areas. The UAE’s National Centre of Meteorology (NCM) has issued a dust storm alert, with varying intensities expected:\nSuspended dust reduced visibility to less than 100 metres in Al Hamra (Al Dhafra) and Umm Al Sheif Island on Tuesday morning. Visibility has been brought down to less than 500 metres on Dalma Island, according to the NCM.\nCountries in the Middle East have been hit by sand storms, with flight operations in two Arab countries – Kuwait and Iraq – affected.\nLast night, the NCM had predicted that a storm brewing over the Arabian Gulf would hit the western part of the UAE today after blowing over Saudi Arabia.\nOn Tuesday, social media handle Storm Centre posted satellite images showing the movement of the dust:\nEarlier today, forecasters predicted severe dusty weather conditions in the UAE with a decrease in temperatures over the next few days.\xa0Windy conditions will per

In [ ]:
for link in website_list:
    for lists in omittedcategories:
        for omit in range(1, len(lists)):
            if lists[omit] in categories:
                categories.remove(lists[omit])
            if os.path.exists(result_path + "/khaleejtimes/texts/"+lists[omit]+".txt"):
                inputFile = open(result_path + "/khaleejtimes/texts/"+lists[omit]+".txt", "r", encoding='utf-8')
                outputFile = open(result_path + "/khaleejtimes/texts/"+lists[0]+".txt", "a", encoding='utf-8')
                text = inputFile.read()
                outputFile.write(text)
                inputFile.close()
                os.remove(result_path + "/khaleejtimes/texts/"+lists[omit]+".txt")


In [ ]:
for link in website_list:
    website_name = link.split('/')[2]
    website_name = website_name.replace("www.","")
    website_name = website_name.replace(".com","")
    path = result_path + '/'+website_name



    for category in categories:
        inputFile = open(result_path + "/khaleejtimes/texts/"+category+".txt", "r", encoding='utf-8')
        TextExtracted = inputFile.read()
        occurences = []
        for word in words:
            occurences.append(TextExtracted.lower().count(word))
        categories_occurences.append(occurences)
        inputFile.close()
    total = [0,0,0,0,0,0]
    for occurence in categories_occurences:
        for i in range(len(occurence)):
            total[i] = total[i] + occurence[i]
    statistics = open(path + "/Statistics.txt", "w", encoding='utf-8')
    CSVstatistics = open(path + "/Statistics.csv", "w", encoding='utf-8')
    csvwriter = csv.writer(CSVstatistics)
    statistics.write("Total occurences:\n")
    for index in range(len(words)):
        statistics.write(words[index]+" = "+ str(total[index])+" occurences.\n")
    statistics.write("Statistics per category:\n")
    statistics.write('{:35s}'.format("Category:"))
    for word in words:
        statistics.write('|' + '{:5s}'.format(word))
    statistics.write("\n")
    for index in range(len(categories)):
        statistics.write('{:35s}'.format(categories[index]))
        for q in range(len(categories_occurences[0])):
            statistics.write('|' + '{:5s}'.format(str(categories_occurences[index][q])))
        statistics.write("\n")
    csvStat = copy.deepcopy(categories_occurences)
    for index in range(len(categories)):
        csvStat[index].append(categories[index])
        csvStat[index] = csvStat[index][-1:] + csvStat[index][:-1]
    csvwriter.writerows(csvStat)
    CSVstatistics.close()
    statistics.close()
